# Setup libraries and working data
Load libraries and set up working directories.

In [1]:
import os
import os.path
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from math import ceil
from random import random
import json

from shapely.geometry import Point
from shapely.geometry import shape
import matplotlib.pyplot as plt
import contextily as ctx
from tqdm.notebook import tqdm
tqdm.pandas()

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from rasterio.features import shapes
from rasterio.features import dataset_features

from pyproj import CRS

%matplotlib inline

DROOT = '../1-data/'
os.makedirs(os.path.join(DROOT, '3-interim', 'populationmasks'), exist_ok=True)

pd.set_option('display.max_columns', None)


# General methods
Define methods to be used in the procedures below.

In [2]:
# Convert single GeoDataFrame Polygon to GeoJSON
def get_mask_coords(gdf): 
    """Get the first polygon in a GeoDataFrame as GeoJSON."""
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [3]:
def mask_popcenter_from_raster(gdf_entry, raster, tiff_out):
    
    out_img, out_transform = mask(
        dataset=raster, 
        shapes=get_mask_coords(gdf_entry), 
        crop=True)

    # Write out for usage a bit further onwards due to limitations in rasterio.
    out_meta = pop.meta.copy()
    out_meta.update({
        "driver": "GTiff",
         "height": out_img.shape[1],
         "width": out_img.shape[2],
         "transform": out_transform,
         "crs": pop.crs
    })

    with rasterio.open(tiff_out, "w", **out_meta) as dest:
        dest.write(out_img)

### Load information files

In [4]:
# Get population data for the whole world.
pop = rasterio.open(os.path.join(
    DROOT,
    '2-external',
    'GHS_POP_E2020_GLOBE_R2022A_54009_1000_V1_0',
    'GHS_POP_E2020_GLOBE_R2022A_54009_1000_V1_0.tif'))
pop.shape, pop.bounds, pop.crs.to_proj4()

((18000, 36082),
 BoundingBox(left=-18041000.0, bottom=-9000000.0, right=18041000.0, top=9000000.0),
 '+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs=True')

In [5]:
# Open Urban Centers file, it's pretty big.
urbancenter_gdf = gpd.read_file( os.path.join(
    DROOT, 
    '2-external',
    'GHS_STAT_UCDB2015MT_GLOBE_R2019A',
    'GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg')).to_crs(pop.crs)
urbancenter_gdf.head(1)

KeyboardInterrupt: 

In [ ]:
name = input('Name: ')
result = urbancenter_gdf[urbancenter_gdf.UC_NM_LST.str.contains(name)]
ax = result.plot()
ctx.add_basemap(ax, crs=result.crs)
result[['CTR_MN_NM', 'CTR_MN_ISO', 'XC_ISO_LST', 'UC_NM_MN', 'UC_NM_LST']]

In [ ]:
# Open Functional Urban Areas file
fua_gdf = gpd.read_file( os.path.join(
    DROOT, 
    '2-external',
    'GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0',
    'GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg')).to_crs(pop.crs)

fua_gdf.head(2)

In [ ]:
# cities = pd.read_excel('../1-data/1-research/cities.xlsx', index_col=0)
# countries = cities['Country'].to_list()
# cities['ID_HDC_G0'] = np.nan
# ids = cities['ID_HDC_G0'].to_list()
# for row in cities.itertuples():
#     result = urbancenter_gdf[(urbancenter_gdf.UC_NM_LST.str.contains(row.City)) & 
#                              (urbancenter_gdf.CTR_MN_ISO == countries[row.Index])]
#     i = 0
#     if len(result) > 1:
#         print(result[['ID_HDC_G0', 'CTR_MN_NM', 'CTR_MN_ISO', 'XC_ISO_LST', 'UC_NM_MN', 'UC_NM_LST']])
#         i = input()
#     ids[row.Index] = result.iloc[int(i)].ID_HDC_G0
    
# # cities['ID_HDC_G0'] = ids
# # cities.to_excel('../1-data/1-research/cities.xlsx')

In [ ]:
name = input('Name (Barcelona / Amsterdam): ')
country = input('Country ISO (ESP / NLD): ')
result = fua_gdf[(fua_gdf.eFUA_name.str.contains(name)) & (fua_gdf.Cntry_ISO == country)]
ax = result.plot()
ctx.add_basemap(ax, crs=result.crs)
result[['eFUA_ID', 'eFUA_name', 'Cntry_ISO', 'FUA_p_2015']]

# Procedures
Load two files and start masking the listed cities.

In [ ]:
image.max()

In [ ]:
plt.imshow(image)

In [ ]:
city_list_df = pd.read_excel(os.path.join(DROOT, '1-research', 'cities.xlsx'))
city_list_itr = tqdm(city_list_df.itertuples(), leave=False, total=len(city_list_df))

os.makedirs(os.path.join(DROOT, '3-interim', 'populationmasks'), exist_ok=True)

# Get masked population dataframes for all mentioned cities.
for city in city_list_itr:
    
    city_list_itr.set_description(f"Current city: {city.City}")

    tiff_path = os.path.join(DROOT, '3-interim', 'populationmasks', 
                             f"{city.ID_HDC_G0}.tiff")
    pcl_path  = os.path.join(DROOT, '3-interim', 'populationmasks', 
                             f"{city.ID_HDC_G0}.pcl")
    
    # If city already done, skip this.
    if(os.path.exists(tiff_path) and os.path.exists(pcl_path)):
        continue;
    
    # Write out a masked selection with city population.
    mask_popcenter_from_raster(
        gdf_entry=urbancenter_gdf[urbancenter_gdf.ID_HDC_G0 == city.ID_HDC_G0],
        raster=pop,
        tiff_out=tiff_path
    )
    
    # Convert the masked tiff to geojson for GeoPandas to use.
    # This is doing the heavy lifting!
    with rasterio.open(tiff_path) as raster:
        image = raster.read(1).astype("float32")
        
        # Add tiny random deviation (<1) so units don't join together
        #   in the next shapes() method. 
        rand_add = np.random.random(image.shape)
        rand_add[image == -200] = 0
        image += rand_add
        
        crs = raster.crs
        list_pop = [
            {'cell_pop': value, 'geometry': shape(shp)}
            for i, (shp, value) 
            in enumerate(shapes(image, transform=raster.transform))
            if value > raster.nodata
        ]
    
    # Read in as a GeoPandas dataset and write out. 
    gdf_pop = gpd.GeoDataFrame(list_pop, crs=crs)
    gdf_pop.cell_pop = np.maximum(gdf_pop.cell_pop, 0)
    gdf_pop.to_pickle(pcl_path)
